In [1]:
import pandas as pd
import numpy as np

In [2]:
d = pd.read_csv(r"C:\Users\ponra\Downloads\archive (3)\Bank_Personal_Loan_Modelling.csv")

In [3]:
d.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [4]:
d.drop('ID',axis=1,inplace=True)

In [5]:
d['CCAvg'] = d['CCAvg'].astype('int64')

In [6]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Age                 5000 non-null   int64
 1   Experience          5000 non-null   int64
 2   Income              5000 non-null   int64
 3   ZIP Code            5000 non-null   int64
 4   Family              5000 non-null   int64
 5   CCAvg               5000 non-null   int64
 6   Education           5000 non-null   int64
 7   Mortgage            5000 non-null   int64
 8   Personal Loan       5000 non-null   int64
 9   Securities Account  5000 non-null   int64
 10  CD Account          5000 non-null   int64
 11  Online              5000 non-null   int64
 12  CreditCard          5000 non-null   int64
dtypes: int64(13)
memory usage: 507.9 KB


In [7]:
d.columns

Index(['Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Personal Loan', 'Securities Account',
       'CD Account', 'Online', 'CreditCard'],
      dtype='object')

In [8]:
d.columns=['Age', 'Experience', 'Income', 'ZIP Code', 'Family', 'CCAvg',
       'Education', 'Mortgage', 'Securities Account',
       'CD Account', 'Online', 'CreditCard','Personal Loan']

In [9]:
x = d.iloc[:,:-1].values
y = d.iloc[:,-1].values

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)

In [11]:
x = x.T

In [12]:
x = np.array(x)
x

array([[-1.77441684, -0.02952359, -0.55299157, ...,  1.54088033,
         1.71536965, -1.51268285],
       [-1.66607847, -0.09633022, -0.44516316, ...,  1.64783451,
         1.73504275, -1.40445376],
       [-0.53822878, -0.8641094 , -1.36379301, ..., -1.08136314,
        -0.53822878,  0.20043396],
       ...,
       [ 2.92891517,  2.92891517, -0.34142334, ..., -0.34142334,
        -0.34142334, -0.34142334],
       [-0.25354028, -0.25354028, -0.25354028, ..., -0.25354028,
        -0.25354028, -0.25354028],
       [-1.21661774, -1.21661774, -1.21661774, ..., -1.21661774,
         0.82195086,  0.82195086]])

In [35]:
def generate():
    w1 = np.random.randn(8,12)
    b1 = np.random.randn(8,1)
    w2 = np.random.randn(1,8)
    b2 = np.random.randn(1,1)
    return w1,b1,w2,b2

def sigmoid(z):
    return 1/(1+np.exp(-z))

def forward(w1,b1,w2,b2,x):
    z1 = w1.dot(x)+b1
    a1 = sigmoid(z1)
    z2 = w2.dot(a1)+b2
    a2 = sigmoid(z2)
    return a2

def accuracy(x,accu,pred,y):
    c = 0
    for i in range(len(pred)):
        if(pred[i] == y[i]):
            c = c + 1
    accu.append(((c/len(y)),x))
    return accu

def crossover_mutation(parent1,parent2):
    childw1 = parent1[0].reshape(1,96)[0]
    childw2 = parent2[0].reshape(1,96)[0]
    ind = np.random.randint(0,96)
    childw1[0:ind],childw2[0:ind] = childw2[0:ind],childw1[0:ind]#cross over
    for i in range(10):
        r = np.random.randint(0,96)
        val1 = np.random.randn()
        val2 = np.random.randn()
        childw1[r] = val1
        childw2[r] = val2
    childw1 = childw1.reshape(8,12)
    childw2 = childw2.reshape(8,12)
    
    childh1 = parent1[1][0]
    childh2 = parent2[1][0]
    indi = np.random.randint(0,8)
    childh1[0:indi],childh2[0:indi] = childh2[0:indi],childh1[0:indi]
    for i in range(3):
        r = np.random.randint(0,8)
        val1 = np.random.randn()
        val2 = np.random.randn()
        childw1[r] = val1
        childw2[r] = val2
    childh1 = childh1.reshape(1,8)
    childh2 = childh2.reshape(1,8)
    return childw1,childh1,childw2,childh2

def fitness(x,y):
    accu = []
    chromos = []
    for i in range(5):
        w1,b1,w2,b2 = generate()
        chromos.append([w1,w2])
        a2 = forward(w1,b1,w2,b2,x)
        pred = a2[0]
        for j in range(len(pred)):
            if(pred[j] >= 0.5):
                pred[j] = 1
            else:
                pred[j] = 0
        accu = accuracy(i,accu,pred,y)
    p1,p2 = choose_population(accu)
    w1,w2,ww1,ww2 = crossover_mutation(chromos[p1],chromos[p2])
    chromos.clear()
    chromos.append([[w1,w2,],[ww1,ww2]])
    return w1,ww1,w2,ww2,b1,b2,chromos
    
    
def choose_population(parent):
    parent = list(parent)
    parent.sort()
    parent = parent[-1:-3:-1]
    return parent[0][1],parent[1][1]

In [36]:
w1,ww1,w2,ww2,b1,b2,chromos = fitness(x,y)

In [44]:
for i in range(100):
    accu=[]
    a1 = forward(w1,b1,w2,b2,x)
    a2 = forward(ww1,b1,ww2,b2,x)
    pred = a2[0]
    for j in range(len(pred)):
        if(pred[j] >= 0.5):
            pred[j] = 1
        else:
            pred[j] = 0
    accu = accuracy(i,accu,pred,y)
    print(accu)
    w1,w2,ww1,ww2 = crossover_mutation(chromos[0][0],chromos[0][1])
    chromos.clear()
    chromos.append([[w1,w2],[ww1,ww2]])

[(0.6458, 0)]
[(0.649, 1)]
[(0.635, 2)]
[(0.703, 3)]
[(0.6268, 4)]
[(0.6186, 5)]
[(0.706, 6)]
[(0.7112, 7)]
[(0.7144, 8)]
[(0.6516, 9)]
[(0.6666, 10)]
[(0.6432, 11)]
[(0.5982, 12)]
[(0.7014, 13)]
[(0.706, 14)]
[(0.6214, 15)]
[(0.6158, 16)]
[(0.706, 17)]
[(0.706, 18)]
[(0.7072, 19)]
[(0.7028, 20)]
[(0.538, 21)]
[(0.6868, 22)]
[(0.706, 23)]
[(0.5342, 24)]
[(0.5444, 25)]
[(0.5906, 26)]
[(0.56, 27)]
[(0.5494, 28)]
[(0.5914, 29)]
[(0.6618, 30)]
[(0.6354, 31)]
[(0.6146, 32)]
[(0.6446, 33)]
[(0.6284, 34)]
[(0.6582, 35)]
[(0.6798, 36)]
[(0.6902, 37)]
[(0.6364, 38)]
[(0.5618, 39)]
[(0.5724, 40)]
[(0.6322, 41)]
[(0.7056, 42)]
[(0.6996, 43)]
[(0.7036, 44)]
[(0.6452, 45)]
[(0.6754, 46)]
[(0.6628, 47)]
[(0.662, 48)]
[(0.635, 49)]
[(0.696, 50)]
[(0.6562, 51)]
[(0.6982, 52)]
[(0.7028, 53)]
[(0.6956, 54)]
[(0.675, 55)]
[(0.7088, 56)]
[(0.7092, 57)]
[(0.6884, 58)]
[(0.5518, 59)]
[(0.56, 60)]
[(0.642, 61)]
[(0.5946, 62)]
[(0.6528, 63)]
[(0.7126, 64)]
[(0.7062, 65)]
[(0.692, 66)]
[(0.6974, 67)]
[(0.667, 